In [1]:
import torch.nn as nn
import torch
from torchinfo import summary
import numpy as np
import os
import coremltools as ct

Torch version 2.2.2 has not been tested with coremltools. You may run into unexpected errors. Torch 2.1.0 is the most recent version that has been tested.


Зададим класс LinearRegressionModel который инициализирует простейшую нейросеть на PyTorch вида линейной модели.

## Класс `LinearRegressionModel`

Этот класс реализует модель линейной регрессии с использованием PyTorch. Он наследует от `nn.Module` и включает базовый линейный слой, который преобразует входные данные из `input_size` в `output_size`.

### Конструктор `__init__`

Инициализирует линейный слой `self.fc`, который принимает размерности входа (`input_size`) и выхода (`output_size`), оба по умолчанию равны 1.

### Метод `forward`

Определяет прямой проход модели, просто возвращая результат применения линейного слоя к входным данным `x`.


In [2]:

class LinearRegressionModel(nn.Module):
    def __init__(self, input_size=1, output_size=1):
        """
        Инициализация простой линейной регрессии.
        """
        super(LinearRegressionModel, self).__init__()
        self.fc = nn.Linear(input_size, output_size)  # Один линейный слой

    def forward(self, x):
        """
        Определение прямого прохода через сеть.
        """
        return self.fc(x)


## Загрузка и подготовка модели к оценке

Загружаем сохраненную модель линейной регрессии из файла `simple_linear_model.pth` с помощью `torch.load`. После загрузки модели вызываем `model.eval()`, чтобы перевести её в режим оценки. Это деактивирует некоторые функции, такие как dropout и пакетная нормализация, обеспечивая консистентное поведение модели при использовании для предсказаний.


In [6]:
model = torch.load("simple_linnear_model.pth")
model.eval()


LinearRegressionModel(
  (fc): Linear(in_features=1, out_features=1, bias=True)
)

In [7]:
print(model.type)

<bound method Module.type of LinearRegressionModel(
  (fc): Linear(in_features=1, out_features=1, bias=True)
)>


## Суммарное описание модели

Используем функцию `summary` для получения детального описания всех слоев модели, включая количество параметров и размеры входных и выходных данных каждого слоя. Это полезно для проверки структуры модели и убеждения в её правильности перед проведением оценки или дополнительным обучением.


In [4]:
summary(model)

Layer (type:depth-idx)                   Param #
LinearRegressionModel                    --
├─Linear: 1-1                            2
Total params: 2
Trainable params: 2
Non-trainable params: 0

## Подготовка входных данных для модели

Создаем тензор `input_data`, содержащий 100 случайно сгенерированных значений, соответствующих нормальному распределению. Каждое значение является одномерным, что соответствует ожидаемому входу модели линейной регрессии. Для проверки содержимого тензора используем `print(torch.tensor(input_data))`, который выводит значения тензора.


In [8]:
input_data = torch.tensor(torch.randn(50,1))
print(input_data)

tensor([[-0.6746],
        [ 1.1635],
        [ 0.7820],
        [-1.4816],
        [-0.9997],
        [-0.1005],
        [-0.3461],
        [ 0.1777],
        [ 0.0600],
        [ 0.2216],
        [ 0.2525],
        [ 1.6965],
        [ 0.8073],
        [ 0.3030],
        [ 0.8693],
        [-0.1410],
        [ 1.0416],
        [ 1.6982],
        [-1.0507],
        [-2.7198],
        [ 0.3462],
        [-1.0171],
        [-0.4846],
        [-0.8639],
        [-0.1523],
        [-0.1681],
        [ 0.6022],
        [-1.7701],
        [ 0.1925],
        [-1.2012],
        [-0.9647],
        [ 0.6328],
        [ 0.5192],
        [ 1.1316],
        [ 0.5697],
        [ 2.3175],
        [ 0.1312],
        [ 0.1161],
        [-0.8289],
        [ 0.4513],
        [-0.1976],
        [ 2.0346],
        [ 0.6108],
        [ 0.6860],
        [ 0.0450],
        [ 1.1327],
        [ 2.0914],
        [ 0.1203],
        [ 1.6327],
        [ 1.0293]])


/var/folders/2m/1nfzrjzx5rjdbx6qbmzt0bjw0000gp/T/ipykernel_75862/452634648.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(torch.randn(50,1))


## Подготовка и использование входных данных для предсказания

### Подготовка входных данных

В этом разделе мы создаем тензор `input_data` с заданными значениями. Эти данные представляют собой вектор признаков, которые будут использоваться для получения предсказаний от модели линейной регрессии. Данные включают в себя различные числовые значения, распределенные в колонке.

### Выполнение предсказаний

Следующий шаг — использование модели для получения предсказаний на основе входных данных. Мы используем контекст `torch.no_grad()`, чтобы отключить расчет градиентов, что является стандартной практикой при делании предсказаний, поскольку это снижает потребление памяти и ускоряет процесс. Затем мы применяем модель к `input_data` и сохраняем результат в переменной `predictions`.



In [27]:
with torch.no_grad():
    predictions = model(input_data)


## Сохранение предсказаний в файл

### Задание пути к файлу

Переменная `file_path` задает имя файла, в который будут сохранены предсказания модели. Имя файла `LIN_значение.txt` указывает на то, что файл содержит значения линейной регрессии.

### Сохранение данных

Используя функцию `np.savetxt` из библиотеки NumPy, сохраняем предсказания модели в текстовый файл. Метод `flatten` применен к массиву `predictions` для преобразования его в одномерный массив, что упрощает запись данных. Аргумент `delimiter=', '` определяет, что значения в файле будут разделены запятой и пробелом.

Этот подход обеспечивает удобное хранение и возможность последующего анализа результатов модели.


In [7]:
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))

file_path = f'{parent_directory}/Files/LIN_значение.txt'

np.savetxt(file_path, predictions.flatten(), delimiter=', ')


## Сохранение модели в формате TorchScript

### Подготовка примера входных данных

Первым шагом является подготовка `input_trace_example`, примера входных данных, который необходим для трассировки модели. В данном случае создается тензор случайных данных размером `(100, 1)`, что соответствует ожидаемым размерам входа модели.

### Трассировка модели

Функция `torch.jit.trace` используется для создания трассированной версии модели. Этот процесс включает выполнение модели с примером входных данных, что позволяет TorchScript записать операции, выполняемые моделью. Результатом является оптимизированная версия модели, которая может быть эффективно выполнена на различных платформах.

### Сохранение трассированной модели

После трассировки модель сохраняется в файл `LIN_model_traced.pt` с использованием `torch.jit.save`. Этот формат файлов TorchScript удобен для последующего развертывания модели в различных средах, включая серверные и мобильные платформы.

Этот подход позволяет улучшить переносимость и производительность модели, обеспечивая возможность её использования в разнообразных приложениях.


In [36]:
input_trase_example = (torch.randn(50,1))

traced_model = torch.jit.trace(model, input_trase_example)
torch.jit.save(traced_model, f'{parent_directory}/Files/LIN_model_traced.pt')

### Загрузка модели LIN
Загружаем модель LIN из файла с использованием формата PyTorch JIT для эффективного выполнения.

### Переключение модели в режим оценки
Переводим модель в режим оценки, что необходимо для деактивации слоёв, используемых только во время обучения, как dropout и batch normalization.


In [37]:
LIN_loaded_model = torch.jit.load(f'{parent_directory}/Files/LIN_model_traced.pt')
LIN_loaded_model.eval()

RecursiveScriptModule(
  original_name=LinearRegressionModel
  (fc): RecursiveScriptModule(original_name=Linear)
)

### Вывод структуры модели LIN
Используем функцию `summary` для отображения подробного описания архитектуры загруженной модели LIN, включая информацию о слоях, их параметрах и выходных размерах.


In [38]:
summary(LIN_loaded_model)

Layer (type:depth-idx)                   Param #
LinearRegressionModel                    --
├─Linear: 1-1                            2
Total params: 2
Trainable params: 2
Non-trainable params: 0

### Создание примера входных данных для модели LIN
Генерируем тензор размером `50x1` с случайными числами, который будет использоваться в качестве входных данных для тестирования модели LIN.


In [43]:
LIN_example_input = torch.randn(50, 1)

### Конвертация модели LIN в формат CoreML
Производим конвертацию модели LIN из PyTorch в формат CoreML, используя `coremltools`. Это позволяет далее использовать модель на устройствах Apple.

### Сохранение модели в формате CoreML
После проверки функционирования модели, сохраняем её в файл `LIN_CoreML.mlmodel` в указанной директории для использования на платформах Apple.


In [44]:
mlmodel = ct.convert(
    LIN_loaded_model,
    inputs=[ct.TensorType(shape=LIN_example_input.shape)],
    convert_to='neuralnetwork'
)

'''Можно запустить модель для проверки перед сохранением'''


# Сохраняем модель в файл
mlmodel.save(f"{parent_directory}/Files/LIN_CoreML.mlmodel")

Support for converting Torch Script Models is experimental. If possible you should use a traced model for conversion.
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 3/3 [00:00<00:00, 13400.33 ops/s]


In [45]:
LIN_coreML = ct.models.MLModel(f'{parent_directory}/Files/LIN_CoreML.mlmodel')

In [46]:
spec = LIN_coreML.get_spec()
print(spec.description.input)
print(spec.description.output)

[name: "x_1"
type {
  multiArrayType {
    shape: 50
    shape: 1
    dataType: FLOAT32
  }
}
]
[name: "linear_0"
type {
  multiArrayType {
    dataType: FLOAT32
  }
}
]


In [47]:
input_d = {'x_1':input_data}
LIN_predict = LIN_coreML.predict(input_d)